# Creating a diagnostics package with CDP

In this notebook, we'll cover the steps needed to create a diagnostics package using the Community Diagnostics Package (CDP) via an example diagnostic package. For a basic diagnostics package, we'll cover how to:
* create metrics
* use them in your main script

Additional points covered will include:
* adding paramaters to your script
* adding command line functionality for the parameters
* optionally adding a viewer

## Overview of CDP

Add some overview?

## Creating metrics

In your project directory, create a folder that will contain all of your metrics. In our example, we'll use a folder titled `my_metrics`.

Our diagnostics will have two metrics `add` and `sub`, which will add and subtract two number respectively. Each metric must have a function called `compute()` and inherit from `CDPMetric`. Below is boilerplate code for creating a metric:

In [ ]:
from cdp.cdp_metric import CDPMetric

class MyMetric(CDPMetric):
    def compute():
        # put your code here

Create two files, `add.py` and `sub.py` in the `my_metrics` folder. Below is the actual code for the metrics:

In [ ]:
# add.py
from cdp.cdp_metric import CDPMetric

class Add(CDPMetric):
    def compute(num1, num2):
        return num1 + num2

In [ ]:
# sub.py
from cdp.cdp_metric import CDPMetric

class Sub(CDPMetric):
    def compute(num1, num2):
        return num1 - num2

So we can import these metrics in other scripts, we need to create an `__init__.py` file in the `my_metrics` folder. We can add the below code in `__init__.py` to make the syntax more clear:

In [ ]:
# my_metrics/__init__.py
import add
import sub

add = add.Add()
sub = sub.Sub()

## Creating your main script

Now all we need to do is import the metrics into your main script.

In [ ]:
from my_metrics import add, sub

print add(1, 1)
print sub(1, 1)

That's all that's needed to create a **very** basic diagnostics package using CDP. The next sections detail how to add more features such as parameters, commandline parsing, and adding in a viewer to view outputs.

## Adding support for parameter files

Scientific software often consists of many variables, all of which might need to be changed for a given run. Software created with CDP allows for parameters to be handled in a more organized way. 